In [54]:
    import fiona
    import os,os.path
    from shapely.geometry import shape,mapping
    from shapely.ops import transform
    from functools import partial
    import pyproj
    from fiona.crs import from_epsg
    import osmnx as ox
    import matplotlib.pyplot as plt
    import geopandas as gpd
    import glob
    import json
    import numpy as np
    import requests as rq
    import collections
    import logging
    import time
    from PIL import Image
    from io import BytesIO
    import pandas as pd
    import dateutil.parser
    from tqdm import tqdm
    %matplotlib inline

In [55]:
# Create a custom logger
logger = logging.getLogger(__name__)

# Create handlers
f_handler = logging.FileHandler('Mapillary_download.log')
f_handler.setLevel(logging.DEBUG)

# Create formatters and add it to handlers
f_format = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
f_handler.setFormatter(f_format)

# Add handlers to the logger
logger.addHandler(f_handler)

In [56]:
city = '../Data/Vienna/pointsJson/'
root = '../Data/Vienna/'
files= glob.glob(city + '*.json')

In [57]:
mapillary_Image_url = 'https://images.mapillary.com/' #<image_key>/thumb-640.jpg

In [58]:
# files

In [60]:
Final_images = {} 
for f in files:
    ImageDict = json.load(open(f,'r'))
    for image in ImageDict:
        if image['properties']['key'] not in Final_images:
            Final_images[image['properties']['key']] = image

In [61]:
# Final_images = json.load(open('../Data/GreaterLondon/pointsJson/All_London_images_mapillary.json','r'))

In [62]:
# ImageDict

In [63]:
len(Final_images.keys())

504051

In [65]:
# json.dump(Final_images,open(city+"All_Vienna_images_mapillary.json",'w'))
# Final_images = json.load(open('../Data/GreaterLondon/pointsJson/All_London_images_mapillary.json'))

In [91]:
def FilterByTime(imageDict , ThreshYear, ThreshMonth ):
    filtered = {}
    for k in imageDict:
        yourdate = dateutil.parser.parse(imageDict[k]['properties']['captured_at'])
        if yourdate.year > ThreshYear and yourdate.month > ThreshMonth:
            filtered[k] = imageDict[k]
    return filtered

def getYearDict(imageDict):
    yearFreq = {}
    for k in imageDict:
        yourdate = dateutil.parser.parse(imageDict[k]['properties']['captured_at'])
        if yourdate.year in yearFreq:
            yearFreq[yourdate.year]+=1
        else:
            yearFreq[yourdate.year]=1
    return yearFreq
        

In [92]:
# Final_images

In [93]:
img_keys = list(Final_images.keys())

In [94]:
Final_images[img_keys[10]]

{'type': 'Feature',
 'properties': {'ca': 111.25839233398438,
  'camera_make': 'ZTE',
  'camera_model': 'ZTE BLADE A0620',
  'captured_at': '2019-01-02T23:17:57.169Z',
  'key': 'e5PBLJ1qHvsLxX0z2cYwgA',
  'pano': False,
  'sequence_key': 'DAnxSBRxT5qDXCsC8LQkXA',
  'user_key': 'a0zJns4SmMKGP5Fm3amhVQ',
  'username': 'hmap_t_181',
  'quality_score': 1},
 'geometry': {'type': 'Point', 'coordinates': [16.4005065, 48.2509761]}}

In [95]:
Final_images[img_keys[10]]['geometry']['coordinates']

[16.4005065, 48.2509761]

In [71]:
yearFreq = getYearDict(Final_images)

In [72]:
yearFreq

{2019: 108353,
 2020: 102491,
 2015: 43011,
 2017: 94881,
 2018: 98770,
 2016: 46024,
 2014: 5945,
 2009: 56,
 2021: 4284,
 2013: 10,
 2008: 76,
 2011: 43,
 2010: 106,
 2006: 1}

In [73]:
pandas_dict = {'Imgkey':[] , 'long': [] , 'lat': [], 'isPano':[] , 'captureTime':[], 'long_lat_string' : [] }  
for k in Final_images:
    pandas_dict['Imgkey'].append(Final_images[k]['properties']['key'])
    pandas_dict['isPano'].append(Final_images[k]['properties']['pano'])
    pandas_dict['captureTime'].append(Final_images[k]['properties']['captured_at'])
    pandas_dict['long'].append(Final_images[k]['geometry']['coordinates'][0])
    pandas_dict['lat'].append(Final_images[k]['geometry']['coordinates'][1])
    coords = Final_images[k]['geometry']['coordinates']
    pandas_dict['long_lat_string'].append(str(format(coords[0], '.4f')) + '_' + str(format(coords[1], '.4f')))
mapillary_df = pd.DataFrame.from_dict(pandas_dict)


In [74]:
# mapillary_df.to_csv("../Data/Vienna/Vienna_Mappilary_points.csv")

# Uncomment below block if you want to download images

In [96]:
def getDownloaded(path):
    filenames = glob.glob(path+'*.jpg')
    print(filenames[1])
    imagekeys = [f.split('/')[-1].split('.')[0] for f in filenames]
    return set(imagekeys)

In [97]:
downloaded = getDownloaded(root + "Mapillary_images/")

../Data/Vienna/Mapillary_images/tPbk1zbzCyA6r5h2HsTCNg.jpg


In [98]:
len(downloaded)

133168

In [ ]:
ImageDir = root + "Mapillary_images/"
for k in tqdm(img_keys):
    ImageKey = Final_images[k]['properties']['key']
    if ImageKey not in downloaded:
        url = mapillary_Image_url + ImageKey + "/thumb-640.jpg"
        logger.debug("Downloading: %s"%(url))
        response = rq.get(url)
        if response.status_code == 200:
            try:
                img = Image.open(BytesIO(response.content))
                filename = '%s/%s.%s' % (ImageDir, ImageKey, 'jpg')
                img.save(filename)
            except:
                logger.debug("Something bad happened while downloading: %s"%(url))
        else:
            logger.debug("Download failed: %s"%(url))



  0%|          | 0/504051 [00:00<?, ?it/s]

  7%|▋         | 33322/504051 [00:00<00:02, 230668.27it/s]

  8%|▊         | 40243/504051 [00:00<00:06, 69080.73it/s] 

 20%|█▉        | 100360/504051 [00:00<00:04, 94054.75it/s]

 26%|██▋       | 133173/504051 [00:00<00:04, 91833.89it/s]

 26%|██▋       | 133173/504051 [00:17<00:04, 91833.89it/s]

 26%|██▋       | 133290/504051 [00:17<4:28:35, 23.01it/s] 

 26%|██▋       | 133291/504051 [00:17<7:42:01, 13.37it/s]

 26%|██▋       | 133292/504051 [00:18<8:44:18, 11.79it/s]

 26%|██▋       | 133293/504051 [00:18<11:06:07,  9.28it/s]

 26%|██▋       | 133294/504051 [00:18<12:25:33,  8.29it/s]

 26%|██▋       | 133295/504051 [00:18<13:20:43,  7.72it/s]

 26%|██▋       | 133296/504051 [00:18<13:24:33,  7.68it/s]

 26%|██▋       | 133297/504051 [00:18<14:11:29,  7.26it/s]

 26%|██▋       | 133298/504051 [00:19<22:32:09,  4.57it/s]

 26%|██▋       | 133299/504051 [00:19<20:43:51,  4.97it/s]

 26%|██▋       | 133300/504051 [00:19<19:39:55,  5.24it/

 26%|██▋       | 133543/504051 [00:54<14:29:59,  7.10it/s]

 26%|██▋       | 133544/504051 [00:55<13:45:13,  7.48it/s]

 26%|██▋       | 133545/504051 [00:55<13:37:46,  7.55it/s]

 26%|██▋       | 133546/504051 [00:55<12:53:48,  7.98it/s]

 26%|██▋       | 133547/504051 [00:55<12:44:29,  8.08it/s]

 26%|██▋       | 133548/504051 [00:55<12:17:16,  8.38it/s]

 26%|██▋       | 133549/504051 [00:55<14:32:06,  7.08it/s]

 26%|██▋       | 133550/504051 [00:55<13:40:20,  7.53it/s]

 26%|██▋       | 133551/504051 [00:56<15:06:31,  6.81it/s]

 26%|██▋       | 133552/504051 [00:56<14:09:21,  7.27it/s]

 26%|██▋       | 133553/504051 [00:56<13:30:08,  7.62it/s]

 26%|██▋       | 133554/504051 [00:56<14:45:32,  6.97it/s]

 26%|██▋       | 133555/504051 [00:56<15:23:39,  6.69it/s]

 26%|██▋       | 133556/504051 [00:56<14:29:39,  7.10it/s]

 26%|██▋       | 133557/504051 [00:56<14:17:41,  7.20it/s]

 26%|██▋       | 133558/504051 [00:56<13:37:03,  7.56it/s]

 26%|██▋       | 133559/504051 [00:57<13

 27%|██▋       | 133796/504051 [01:31<13:35:52,  7.56it/s]

 27%|██▋       | 133797/504051 [01:32<13:56:46,  7.37it/s]

 27%|██▋       | 133798/504051 [01:32<13:57:13,  7.37it/s]

 27%|██▋       | 133799/504051 [01:32<15:10:09,  6.78it/s]

 27%|██▋       | 133800/504051 [01:32<14:12:39,  7.24it/s]

 27%|██▋       | 133801/504051 [01:32<15:23:40,  6.68it/s]

 27%|██▋       | 133802/504051 [01:32<15:05:45,  6.81it/s]

 27%|██▋       | 133803/504051 [01:33<18:37:03,  5.52it/s]

 27%|██▋       | 133804/504051 [01:33<17:56:21,  5.73it/s]

 27%|██▋       | 133805/504051 [01:33<17:25:50,  5.90it/s]

 27%|██▋       | 133807/504051 [01:33<16:02:18,  6.41it/s]

 27%|██▋       | 133808/504051 [01:33<16:18:22,  6.31it/s]

 27%|██▋       | 133809/504051 [01:33<15:56:32,  6.45it/s]

 27%|██▋       | 133810/504051 [01:34<15:27:19,  6.65it/s]

 27%|██▋       | 133811/504051 [01:34<14:23:34,  7.15it/s]

 27%|██▋       | 133812/504051 [01:34<14:41:35,  7.00it/s]

 27%|██▋       | 133813/504051 [01:34<14

 27%|██▋       | 134050/504051 [02:07<15:07:33,  6.79it/s]

 27%|██▋       | 134051/504051 [02:08<15:05:58,  6.81it/s]

 27%|██▋       | 134052/504051 [02:08<15:37:26,  6.58it/s]

 27%|██▋       | 134053/504051 [02:08<16:10:42,  6.35it/s]

 27%|██▋       | 134054/504051 [02:08<17:28:59,  5.88it/s]

 27%|██▋       | 134055/504051 [02:08<16:47:03,  6.12it/s]

 27%|██▋       | 134056/504051 [02:08<16:48:14,  6.12it/s]

 27%|██▋       | 134057/504051 [02:09<16:55:00,  6.08it/s]

 27%|██▋       | 134058/504051 [02:09<16:53:59,  6.08it/s]

 27%|██▋       | 134059/504051 [02:09<16:41:46,  6.16it/s]

 27%|██▋       | 134060/504051 [02:09<16:25:56,  6.25it/s]

 27%|██▋       | 134061/504051 [02:09<16:22:01,  6.28it/s]

 27%|██▋       | 134062/504051 [02:09<16:30:04,  6.23it/s]

 27%|██▋       | 134063/504051 [02:09<15:21:07,  6.69it/s]

 27%|██▋       | 134064/504051 [02:10<17:34:22,  5.85it/s]

 27%|██▋       | 134065/504051 [02:10<15:47:53,  6.51it/s]

 27%|██▋       | 134066/504051 [02:10<16

 27%|██▋       | 134299/504051 [02:45<14:00:06,  7.34it/s]

 27%|██▋       | 134300/504051 [02:45<14:24:29,  7.13it/s]

 27%|██▋       | 134301/504051 [02:45<13:28:30,  7.62it/s]

 27%|██▋       | 134302/504051 [02:45<12:48:01,  8.02it/s]

 27%|██▋       | 134303/504051 [02:46<13:44:19,  7.48it/s]

 27%|██▋       | 134304/504051 [02:46<13:00:33,  7.89it/s]

 27%|██▋       | 134305/504051 [02:46<12:37:01,  8.14it/s]

 27%|██▋       | 134306/504051 [02:46<12:18:39,  8.34it/s]

 27%|██▋       | 134307/504051 [02:46<11:58:04,  8.58it/s]

 27%|██▋       | 134308/504051 [02:46<11:36:08,  8.85it/s]

 27%|██▋       | 134309/504051 [02:46<12:46:48,  8.04it/s]

 27%|██▋       | 134310/504051 [02:46<13:19:31,  7.71it/s]

 27%|██▋       | 134311/504051 [02:47<15:21:37,  6.69it/s]

 27%|██▋       | 134312/504051 [02:47<14:12:16,  7.23it/s]

 27%|██▋       | 134313/504051 [02:47<13:03:41,  7.86it/s]

 27%|██▋       | 134314/504051 [02:47<13:19:34,  7.71it/s]

 27%|██▋       | 134315/504051 [02:47<13

...

 34%|███▍      | 171468/504051 [1:33:03<16:39:33,  5.55it/s]

 34%|███▍      | 171469/504051 [1:33:04<16:48:57,  5.49it/s]

 34%|███▍      | 171470/504051 [1:33:04<14:46:42,  6.25it/s]

 34%|███▍      | 171471/504051 [1:33:04<15:09:43,  6.09it/s]

 34%|███▍      | 171472/504051 [1:33:04<14:49:52,  6.23it/s]

 34%|███▍      | 171473/504051 [1:33:04<13:39:08,  6.77it/s]

 34%|███▍      | 171474/504051 [1:33:04<12:44:31,  7.25it/s]

 34%|███▍      | 171475/504051 [1:33:04<12:38:04,  7.31it/s]

 34%|███▍      | 171476/504051 [1:33:05<12:26:54,  7.42it/s]

 34%|███▍      | 171477/504051 [1:33:05<13:18:17,  6.94it/s]

 34%|███▍      | 171478/504051 [1:33:05<12:31:39,  7.37it/s]

 34%|███▍      | 171479/504051 [1:33:05<12:24:18,  7.45it/s]

 34%|███▍      | 171480/504051 [1:33:05<13:02:48,  7.08it/s]

 34%|███▍      | 171481/504051 [1:33:05<13:09:05,  7.02it/s]

 34%|███▍      | 171482/504051 [1:33:06<15:12:40,  6.07it/s]

 34%|███▍      | 171483/504051 [1:33:06<14:39:24,  6.30it/s]

 34%|███

 34%|███▍      | 171722/504051 [1:33:38<14:38:59,  6.30it/s]

 34%|███▍      | 171723/504051 [1:33:39<13:33:08,  6.81it/s]

 34%|███▍      | 171724/504051 [1:33:39<12:45:08,  7.24it/s]

 34%|███▍      | 171725/504051 [1:33:39<12:03:43,  7.65it/s]

 34%|███▍      | 171726/504051 [1:33:39<12:30:06,  7.38it/s]

 34%|███▍      | 171727/504051 [1:33:39<11:56:25,  7.73it/s]

 34%|███▍      | 171728/504051 [1:33:39<12:47:37,  7.22it/s]

 34%|███▍      | 171729/504051 [1:33:39<12:47:22,  7.22it/s]

 34%|███▍      | 171730/504051 [1:33:39<12:05:23,  7.64it/s]

 34%|███▍      | 171731/504051 [1:33:40<13:44:10,  6.72it/s]

 34%|███▍      | 171732/504051 [1:33:40<14:38:30,  6.30it/s]

 34%|███▍      | 171733/504051 [1:33:40<15:12:54,  6.07it/s]

 34%|███▍      | 171734/504051 [1:33:40<13:49:03,  6.68it/s]

 34%|███▍      | 171735/504051 [1:33:40<14:24:16,  6.41it/s]

 34%|███▍      | 171736/504051 [1:33:40<13:29:05,  6.85it/s]

 34%|███▍      | 171737/504051 [1:33:41<12:35:35,  7.33it/s]

 34%|███

 34%|███▍      | 171975/504051 [1:34:12<11:33:16,  7.98it/s]

 34%|███▍      | 171977/504051 [1:34:12<11:27:05,  8.06it/s]

 34%|███▍      | 171978/504051 [1:34:12<11:13:03,  8.22it/s]

 34%|███▍      | 171979/504051 [1:34:12<12:11:37,  7.56it/s]

 34%|███▍      | 171980/504051 [1:34:13<12:41:43,  7.27it/s]

 34%|███▍      | 171981/504051 [1:34:13<13:02:35,  7.07it/s]

 34%|███▍      | 171982/504051 [1:34:13<13:10:16,  7.00it/s]

 34%|███▍      | 171983/504051 [1:34:13<13:13:18,  6.98it/s]

 34%|███▍      | 171984/504051 [1:34:13<13:24:26,  6.88it/s]

 34%|███▍      | 171985/504051 [1:34:13<14:11:50,  6.50it/s]

 34%|███▍      | 171986/504051 [1:34:13<14:00:21,  6.59it/s]

 34%|███▍      | 171987/504051 [1:34:14<13:47:58,  6.68it/s]

 34%|███▍      | 171988/504051 [1:34:14<14:18:50,  6.44it/s]

 34%|███▍      | 171990/504051 [1:34:14<13:23:42,  6.89it/s]

 34%|███▍      | 171991/504051 [1:34:14<12:40:57,  7.27it/s]

 34%|███▍      | 171992/504051 [1:34:14<14:08:58,  6.52it/s]

 34%|███

 34%|███▍      | 172238/504051 [1:34:47<10:02:55,  9.17it/s]

 34%|███▍      | 172239/504051 [1:34:47<10:37:22,  8.68it/s]

 34%|███▍      | 172240/504051 [1:34:47<10:43:46,  8.59it/s]

 34%|███▍      | 172241/504051 [1:34:48<10:38:35,  8.66it/s]

 34%|███▍      | 172242/504051 [1:34:48<11:56:19,  7.72it/s]

 34%|███▍      | 172243/504051 [1:34:48<12:36:33,  7.31it/s]

 34%|███▍      | 172244/504051 [1:34:48<13:37:58,  6.76it/s]

 34%|███▍      | 172245/504051 [1:34:48<14:19:52,  6.43it/s]

 34%|███▍      | 172246/504051 [1:34:48<13:55:30,  6.62it/s]

 34%|███▍      | 172247/504051 [1:34:49<14:03:31,  6.56it/s]

 34%|███▍      | 172248/504051 [1:34:49<14:17:39,  6.45it/s]

 34%|███▍      | 172249/504051 [1:34:49<14:17:20,  6.45it/s]

 34%|███▍      | 172250/504051 [1:34:49<13:53:32,  6.63it/s]

 34%|███▍      | 172251/504051 [1:34:49<14:03:14,  6.56it/s]

 34%|███▍      | 172252/504051 [1:34:49<13:51:06,  6.65it/s]

 34%|███▍      | 172253/504051 [1:34:49<13:50:54,  6.66it/s]

 34%|███

 34%|███▍      | 172487/504051 [1:35:22<12:38:34,  7.28it/s]

 34%|███▍      | 172488/504051 [1:35:22<12:24:43,  7.42it/s]

 34%|███▍      | 172489/504051 [1:35:22<13:35:52,  6.77it/s]

 34%|███▍      | 172490/504051 [1:35:22<12:31:55,  7.35it/s]

 34%|███▍      | 172491/504051 [1:35:22<11:41:18,  7.88it/s]

 34%|███▍      | 172492/504051 [1:35:22<11:59:09,  7.68it/s]

 34%|███▍      | 172493/504051 [1:35:23<11:42:53,  7.86it/s]

 34%|███▍      | 172494/504051 [1:35:23<12:31:01,  7.36it/s]

 34%|███▍      | 172495/504051 [1:35:23<12:04:47,  7.62it/s]

 34%|███▍      | 172496/504051 [1:35:23<13:40:21,  6.74it/s]

 34%|███▍      | 172497/504051 [1:35:23<14:36:52,  6.30it/s]

 34%|███▍      | 172498/504051 [1:35:23<16:54:38,  5.45it/s]

 34%|███▍      | 172499/504051 [1:35:24<16:31:04,  5.58it/s]

 34%|███▍      | 172500/504051 [1:35:24<16:15:22,  5.67it/s]

 34%|███▍      | 172501/504051 [1:35:24<16:17:58,  5.65it/s]

 34%|███▍      | 172502/504051 [1:35:24<16:29:33,  5.58it/s]

 34%|███

In [75]:
len(mapillary_df['long_lat_string'].unique())

163587

In [52]:
mapillary_df.head()

,Imgkey,long,lat,isPano,captureTime,long_lat_string
0,SNkE9dXx2K0o3MaY9Zu-2A,-0.178409,51.523651,False,2016-03-24T14:44:50.927Z,-0.1784_51.5237
1,bneiFOYn9N2MQpjD-E7y-Q,-0.036538,51.452568,False,2018-05-03T10:59:33.785Z,-0.0365_51.4526
2,C5QMwGz01Rs6zk56q6fkeA,-0.389959,51.462317,False,2018-12-09T15:42:45.172Z,-0.3900_51.4623
3,YBQNM1wNdihc3tl1Jr1oDA,-0.095355,51.508081,False,2014-08-04T10:19:53.000Z,-0.0954_51.5081
4,6iT4CuOAHfSYNXWs_hlbdw,-0.120967,51.485719,False,2015-11-25T13:50:11.000Z,-0.1210_51.4857
